# Task3. 酶的EC号预测

> author: Shizhenkun   
> email: zhenkun.shi@tib.cas.cn   
> date: 2021-09-09  

## 任务简介
该任务通过给定酶序列，预测该酶的反应类别（EC号）






## 1. 导入必要的包

In [2]:
import numpy as np
import pandas as pd
import sys
import os
from tqdm import tqdm
sys.path.append("../../tools/")
import commontools
import funclib

from xgboost.sklearn import XGBClassifier


from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import hamming_loss
from sklearn.metrics import jaccard_score
from sklearn.metrics import hinge_loss

sys.path.append("../../")
import benchmark_common as bcommon
import benchmark_train as btrain
import benchmark_test as btest
import benchmark_config as cfg
import benchmark_evaluation as eva

%load_ext autoreload
%autoreload 2

## 2. 加载数据

In [3]:
# 1. 读入数据
print('step 1 loading data')
train = pd.read_feather(cfg.TRAIN_FEATURE)
test = pd.read_feather(cfg.TEST_FEATURE)
# train,test= bcommon.load_data_embedding(train=train, test=test, embedding_type=cfg.EMBEDDING_METHOD.get('unirep'))

step 1 loading data


## 3. 同源比对

In [3]:
#写fasta文件
funclib.table2fasta(train, '/tmp/train.fasta')
funclib.table2fasta(test, '/tmp/test.fasta')
# 读取比对结果
res_data=funclib.getblast_fasta(trainfasta='/tmp/train.fasta',testfasta='/tmp/test.fasta')
print(' query samples:{0}\n results samples: {1}'.format(len(test), len(res_data)))

Write finished
Write finished
diamond makedb --in /tmp/train.fasta -d /tmp/train.dmnd
diamond blastp -d /tmp/train.dmnd  -q  /tmp/test.fasta -o /tmp/test_fasta_results.tsv -b5 -c1 -k 1
 query samples:8033
 results samples: 6034


In [4]:
# 提取测试集项目
test_results_df=test[['id', 'ec_number']]

# 给预测结果添加EC
id_ec_dict = {v: k for v,k in zip( train.id, train.ec_number)} 
res_data['ec_pred'] = res_data.sseqid.apply(lambda x : id_ec_dict.get(x))

pred_ec_dict = {v: k for v,k in zip( res_data.id, res_data.ec_pred)} 
with pd.option_context('mode.chained_assignment', None):
    test_results_df['ec_pred_blast'] = test_results_df.id.apply(lambda x: pred_ec_dict.get(x))
    
 #4. 加载EC号训练数据
print('loading ec to label dict')
if os.path.exists(cfg.FILE_EC_LABEL_DICT):
    dict_ec_label = np.load(cfg.FILE_EC_LABEL_DICT, allow_pickle=True).item()
else:
    dict_ec_label = btrain.make_ec_label(train_label=train['ec_number'], test_label=test['ec_number'], file_save= cfg.FILE_EC_LABEL_DICT, force_model_update=cfg.UPDATE_MODEL)

In [5]:
test_results_df[(test_results_df.ec_number==test_results_df.ec_pred_blast) & (test_results_df.ec_number!='-')]


loading ec to label dict


In [6]:
test_results_df

,id,ec_number,ec_pred_blast
0,O02324,-,-
1,P0DQF6,-,None
2,P0DQF5,-,None
3,P0DQF4,-,None
4,P0DQF3,-,None
...,...,...,...
8028,Q9NTI2,7.6.2.1,3.6.3.1
8029,D4AA47,7.6.2.1,3.6.3.1
8030,F1RBC8,7.6.2.4,-
8031,Q7JUN3,7.6.2.4,-


## 4. Slice+Unirep

In [6]:
#训练数据
train_data = train[(train.functionCounts==1) & (train.ec_specific_level>=2)]  #用酶->EC号大于2级
train_data.reset_index(drop=True, inplace=True)

#添加标签
train_data.insert(loc=1, column='ec_label', value=train_data.ec_number.apply(lambda x: dict_ec_label.get(x))) 

train_X = train_data.iloc[:, 8:]
train_Y = pd.DataFrame(train_data['ec_label'])

In [ ]:
#试数据
test.insert(loc=1, column='ec_label', value=test.ec_number.apply(lambda x: dict_ec_label.get(x)))
test_X = test.iloc[:, 8:]
test_Y = pd.DataFrame(test['ec_label'])

In [57]:
#train
bcommon.prepare_slice_file(x_data=train_X, y_data=train_Y, x_file=cfg.DATADIR+'slice_train_x_unirep.txt', y_file=cfg.DATADIR+'slice_train_y_unirep.txt', ec_label_dict=dict_ec_label)
#test
bcommon.prepare_slice_file(x_data=test_X, y_data=test_Y, x_file=cfg.DATADIR+'slice_test_x_unirep.txt', y_file=cfg.DATADIR+'slice_test_y_unirep.txt', ec_label_dict=dict_ec_label)

slice files prepared success


In [10]:
print('step 6 trainning slice model')
#6. 训练Slice模型
btrain.train_ec_slice(trainX=cfg.DATADIR+'slice_train_x_unirep.txt', trainY=cfg.DATADIR+'slice_train_y_unirep.txt', modelPath=cfg.MODELDIR+'/slice_unirep')

step 6 trainning slice model
 ./slice_train /home/shizhenkun/codebase/BioUniprot/data/benchmark/data/slice_train_x_unirep.txt /home/shizhenkun/codebase/BioUniprot/data/benchmark/data/slice_train_y_unirep.txt /home/shizhenkun/codebase/BioUniprot/data/benchmark/model/slice_unirep -m 100 -c 300 -s 300 -k 700 -o 32 -t 32 -C 1 -f 0.000001 -siter 20 -stype 0 -q 0 
train finished


In [76]:
slice_pred = btest.get_slice_res(slice_query_file=cfg.DATADIR+'slice_test_x_unirep.txt', model_path= cfg.MODELDIR+'/slice_unirep',dict_ec_label=dict_ec_label,test_set=test, res_file='/tmp/test.txt')

./slice_predict /home/shizhenkun/codebase/BioUniprot/data/benchmark/data/slice_test_x_unirep.txt /home/shizhenkun/codebase/BioUniprot/data/benchmark/model/slice_unirep /tmp/test.txt -o 32 -b 0 -t 32 -q 0


In [181]:
s1res = test_results_df.merge(slice_pred, on='id', how='left')

In [183]:
s1res[(s1res.ec_number==s1res.top0) &(s1res.ec_number!=s1res.ec_pred_blast)]

,id,ec_number,ec_pred_blast,top0,top1,top2,top3,top4,top5,top6,...,top10,top11,top12,top13,top14,top15,top16,top17,top18,top19
4852,A0A3B1EFP9,1.1.-.-,1.1.99.32,1.1.-.-,1.1.99.29,1.1.3.10,1.1.3.13,4.1.2.10,3.2.1.21,1.14.13.-,...,1.14.14.-,1.1.3.37,1.4.3.4,1.1.1.102,1.1.5.9,1.1.99.-,1.4.3.16,3.4.16.4,1.3.5.4,1.1.99.2
4870,A0A1Y0BRF3,1.1.1.-,1.1.1.2,1.1.1.-,1.1.1.1,1.3.1.-,5.1.3.2,1.3.1.112,1.1.1.2,1.5.1.46,...,1.1.1.170,1.4.3.3,1.1.1.349,1.1.1.283,1.2.1.44,1.1.1.270,1.3.1.3,1.8.-.-,1.3.1.77,1.14.13.237
4873,M1VWN4,1.1.1.-,1.-.-.-,1.1.1.-,2.3.1.-,1.1.1.330,1.1.1.102,1.3.1.-,1.16.1.-,1.1.1.170,...,2.7.1.-,1.2.1.-,1.2.1.84,1.6.5.9,1.1.5.3,1.3.1.45,1.5.1.7,1.3.1.75,1.5.1.40,1.16.1.9
4874,D7UTD0,1.1.1.-,1.-.-.-,1.1.1.-,1.1.1.330,1.1.1.1,1.3.1.33,1.1.1.270,1.1.-.-,1.1.1.307,...,3.1.-.-,1.1.1.298,1.3.1.-,1.1.1.2,1.1.1.101,1.1.1.138,2.1.1.261,1.1.3.4,1.1.1.283,1.1.1.332
4877,A0A2H3CNT9,1.1.1.-,1.-.-.-,1.1.1.-,1.1.1.330,1.3.1.33,1.1.-.-,1.1.1.1,1.1.1.270,1.1.1.307,...,4.4.1.5,1.1.1.349,1.14.14.-,1.1.1.138,1.1.1.2,1.1.1.101,1.3.1.-,1.1.3.13,1.3.1.34,1.1.1.283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7510,S3E7P8,4.2.1.33,4.2.1.-,4.2.1.33,4.2.1.36,2.3.3.16,3.5.1.9,6.5.1.3,4.2.1.109,2.7.2.4,...,4.1.1.19,2.3.1.37,4.2.1.85,3.1.3.99,1.7.1.4,1.1.1.34,1.2.1.51,5.4.3.10,4.2.1.42,2.7.1.220
7513,A0A2H3E4G0,4.2.1.66,3.5.5.1,4.2.1.66,2.3.1.-,3.5.5.1,3.5.1.4,3.5.1.49,3.5.1.-,6.3.5.1,...,3.5.5.7,3.5.1.6,3.5.1.92,3.5.1.53,3.5.1.12,3.6.-.-,3.5.1.77,3.1.13.4,1.2.4.1,1.14.14.-
7669,A0A3G1DJK5,4.3.1.24,5.4.3.10,4.3.1.24,4.3.1.3,4.1.1.31,5.3.1.9,1.3.3.6,1.17.4.1,4.3.1.25,...,2.5.1.54,1.4.1.2,4.1.1.6,1.1.1.34,5.4.3.10,1.5.99.12,1.1.3.37,1.1.1.67,4.3.1.23,4.2.1.79
7849,F2RB81,6.2.1.-,5.1.1.-,6.2.1.-,6.3.2.26,2.3.1.-,1.1.1.329,6.3.2.-,4.2.3.124,1.14.13.181,...,2.3.1.246,6.2.1.20,1.1.1.364,3.5.4.40,2.4.1.285,3.5.2.19,2.4.1.283,6.2.1.46,1.1.1.340,4.1.1.-


## 5. Slice+esm0

In [111]:
train_esm0,test_esm0= bcommon.load_data_embedding(train=train_data, test=test, embedding_type=cfg.EMBEDDING_METHOD.get('esm0'))

In [119]:

train_X = train_esm0.iloc[:, 7:]
train_Y = pd.DataFrame(train_esm0['ec_label'])

test_X = test_esm0.iloc[:, 7:]
test_Y = pd.DataFrame(test_esm0['ec_label'])

In [121]:
cfg.FEATURE_NUM = 1280
#train
bcommon.prepare_slice_file(x_data=train_X, y_data=train_Y, x_file=cfg.DATADIR+'slice_train_x_esm0.txt', y_file=cfg.DATADIR+'slice_train_y_esm0.txt', ec_label_dict=dict_ec_label)
#test
bcommon.prepare_slice_file(x_data=test_X, y_data=test_Y, x_file=cfg.DATADIR+'slice_test_x_esm0.txt', y_file=cfg.DATADIR+'slice_test_y_esm0.txt', ec_label_dict=dict_ec_label)

slice files prepared success
slice files prepared success


In [122]:
print('step 6 trainning slice model')
#6. 训练Slice模型
btrain.train_ec_slice(trainX=cfg.DATADIR+'slice_train_x_esm0.txt', trainY=cfg.DATADIR+'slice_train_y_unirep.txt', modelPath=cfg.MODELDIR+'/slice_esm0')

step 6 trainning slice model
 ./slice_train /home/shizhenkun/codebase/BioUniprot/data/benchmark/data/slice_train_x_esm0.txt /home/shizhenkun/codebase/BioUniprot/data/benchmark/data/slice_train_y_unirep.txt /home/shizhenkun/codebase/BioUniprot/data/benchmark/model/slice_esm0 -m 100 -c 300 -s 300 -k 700 -o 32 -t 32 -C 1 -f 0.000001 -siter 20 -stype 0 -q 0 
train finished


In [123]:
slice_pred = btest.get_slice_res(slice_query_file=cfg.DATADIR+'slice_test_x_esm0.txt', model_path= cfg.MODELDIR+'/slice_esm0',dict_ec_label=dict_ec_label,test_set=test, res_file='/tmp/test.txt')

./slice_predict /home/shizhenkun/codebase/BioUniprot/data/benchmark/data/slice_test_x_esm0.txt /home/shizhenkun/codebase/BioUniprot/data/benchmark/model/slice_esm0 /tmp/test.txt -o 32 -b 0 -t 32 -q 0


In [127]:
s1res = test.iloc[:,np.r_[0:5]].merge(slice_pred, on='id', how='left')
s1res[s1res.ec_number==s1res.top0]

,id,ec_label,isemzyme,functionCounts,ec_number,top0,top1,top2,top3,top4,...,top10,top11,top12,top13,top14,top15,top16,top17,top18,top19
4970,P28332,35.0,True,1,1.1.1.1,1.1.1.1,6.3.2.4,2.1.1.195,2.3.1.74,1.1.1.8,...,1.7.1.7,2.7.9.3,4.1.3.4,1.17.7.3,2.6.1.37,4.2.3.4,2.1.1.68,1.1.1.14,4.2.1.93,1.3.1.104
4990,W6QIT2,114.0,True,1,1.1.1.205,1.1.1.205,3.6.3.14,1.1.5.4,5.3.1.9,5.4.2.12,...,1.3.3.4,5.4.2.3,2.7.1.40,2.7.11.17,3.1.2.1,1.6.2.6,1.14.14.17,1.1.2.4,2.7.1.17,6.5.1.3
5004,A0A0S3QTC6,253.0,True,1,1.1.1.37,1.1.1.37,2.1.2.9,2.7.2.8,2.2.1.2,1.1.1.27,...,2.7.2.2,3.2.2.8,3.1.3.25,6.3.2.4,1.3.1.-,1.3.1.84,1.1.1.-,1.1.1.237,1.1.1.299,1.1.1.215
5031,C9K7B7,339.0,True,1,1.1.1.85,1.1.1.85,3.1.1.61,1.17.7.3,2.1.1.195,1.1.1.267,...,3.5.1.18,1.1.-.-,4.2.1.93,4.2.1.24,3.5.2.15,5.5.1.7,1.1.1.12,2.7.1.2,1.1.3.15,2.3.1.246
5033,S3DB74,339.0,True,1,1.1.1.85,1.1.1.85,2.3.1.234,3.5.1.18,2.5.1.46,4.2.1.24,...,3.2.1.39,5.5.1.7,2.3.3.14,1.1.1.261,2.1.1.216,1.2.1.11,2.1.1.195,1.5.3.-,1.1.3.15,4.2.1.171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7754,T2KN98,5606.0,True,1,5.3.1.17,5.3.1.17,3.6.3.27,3.5.99.6,4.3.3.7,4.1.2.19,...,5.3.1.30,2.6.1.21,1.1.1.25,2.7.1.176,2.3.1.204,3.1.21.1,1.1.1.250,3.4.22.56,3.4.24.67,1.13.11.49
7755,Q2RXI0,5608.0,True,1,5.3.1.23,5.3.1.23,3.5.1.18,2.7.1.170,2.5.1.6,3.5.2.7,...,1.1.1.1,5.1.1.1,3.5.3.12,4.2.1.8,2.7.9.3,1.7.2.1,1.4.1.3,1.12.99.6,2.6.1.81,1.4.99.-
7951,G0HV10,5977.0,True,1,6.3.4.2,6.3.4.2,6.1.1.6,6.3.5.2,5.4.2.12,3.5.4.2,...,1.8.1.2,6.1.1.23,4.1.1.49,3.4.17.-,3.2.1.141,6.3.5.4,6.1.1.12,6.2.1.-,2.3.3.9,6.2.1.12
7952,O66164,5983.0,True,1,6.3.4.3,6.3.4.3,4.2.1.9,4.2.1.49,5.3.1.9,6.1.1.19,...,2.3.3.9,5.4.2.2,1.16.1.1,2.7.1.16,4.1.1.8,2.2.1.9,1.1.1.38,1.2.1.24,4.1.1.74,2.7.1.29


## 6. Slice+esm32

In [132]:
train_esm32,test_esm32= bcommon.load_data_embedding(train=train_data, test=test, embedding_type=cfg.EMBEDDING_METHOD.get('esm32'))

In [133]:
train_X = train_esm32.iloc[:, 7:]
train_Y = pd.DataFrame(train_esm32['ec_label'])

test_X = test_esm32.iloc[:, 7:]
test_Y = pd.DataFrame(test_esm32['ec_label'])

In [134]:
cfg.FEATURE_NUM = 1280
#train
bcommon.prepare_slice_file(x_data=train_X, y_data=train_Y, x_file=cfg.DATADIR+'slice_train_x_esm32.txt', y_file=cfg.DATADIR+'slice_train_y_esm32.txt', ec_label_dict=dict_ec_label)
#test
bcommon.prepare_slice_file(x_data=test_X, y_data=test_Y, x_file=cfg.DATADIR+'slice_test_x_esm32.txt', y_file=cfg.DATADIR+'slice_test_y_esm32.txt', ec_label_dict=dict_ec_label)

slice files prepared success
slice files prepared success


In [135]:
print('step 6 trainning slice model')
#6. 训练Slice模型
btrain.train_ec_slice(trainX=cfg.DATADIR+'slice_train_x_esm32.txt', trainY=cfg.DATADIR+'slice_train_y_unirep.txt', modelPath=cfg.MODELDIR+'/slice_esm32')
slice_pred = btest.get_slice_res(slice_query_file=cfg.DATADIR+'slice_test_x_esm32.txt', model_path= cfg.MODELDIR+'/slice_esm32',dict_ec_label=dict_ec_label,test_set=test, res_file='/tmp/test.txt')

step 6 trainning slice model
 ./slice_train /home/shizhenkun/codebase/BioUniprot/data/benchmark/data/slice_train_x_esm32.txt /home/shizhenkun/codebase/BioUniprot/data/benchmark/data/slice_train_y_unirep.txt /home/shizhenkun/codebase/BioUniprot/data/benchmark/model/slice_esm32 -m 100 -c 300 -s 300 -k 700 -o 32 -t 32 -C 1 -f 0.000001 -siter 20 -stype 0 -q 0 
train finished
./slice_predict /home/shizhenkun/codebase/BioUniprot/data/benchmark/data/slice_test_x_esm32.txt /home/shizhenkun/codebase/BioUniprot/data/benchmark/model/slice_esm32 /tmp/test.txt -o 32 -b 0 -t 32 -q 0


In [136]:
s1res = test.iloc[:,np.r_[0:5]].merge(slice_pred, on='id', how='left')
s1res[s1res.ec_number==s1res.top0]

,id,ec_label,isemzyme,functionCounts,ec_number,top0,top1,top2,top3,top4,...,top10,top11,top12,top13,top14,top15,top16,top17,top18,top19
4852,A0A3B1EFP9,13.0,True,1,1.1.-.-,1.1.-.-,1.1.99.29,4.1.2.10,3.2.1.21,1.11.1.16,...,1.1.99.1,1.4.3.16,1.1.3.4,1.1.5.4,1.1.1.-,1.1.3.20,1.4.3.4,1.11.1.5,1.14.14.-,1.1.5.3
4870,A0A1Y0BRF3,18.0,True,1,1.1.1.-,1.1.1.-,1.3.1.-,1.1.1.1,5.1.3.2,2.4.1.-,...,1.2.1.38,1.1.1.2,1.2.1.44,1.3.1.33,1.1.1.270,1.1.1.170,1.5.1.46,4.2.1.47,1.4.3.3,1.3.1.45
4871,A0A1A9TAK5,18.0,True,1,1.1.1.-,1.1.1.-,1.1.1.1,1.1.1.100,1.1.-.-,3.1.-.-,...,1.1.1.138,1.1.1.300,1.3.1.38,1.1.1.236,2.7.10.2,1.1.1.153,1.1.1.36,1.1.1.141,1.1.1.30,1.1.1.332
4872,Q5M8N4,18.0,True,1,1.1.1.-,1.1.1.-,2.1.1.163,1.4.99.-,2.4.1.227,5.1.3.2,...,1.1.1.133,1.2.1.84,1.1.1.170,1.1.1.271,1.1.1.37,2.7.-.-,1.3.3.4,2.3.1.20,1.17.1.8,1.2.1.-
4873,M1VWN4,18.0,True,1,1.1.1.-,1.1.1.-,2.3.1.-,1.17.1.8,1.1.1.330,1.1.-.-,...,1.6.5.9,1.1.5.3,1.1.1.270,1.18.1.-,2.7.1.-,1.3.1.75,1.5.1.46,3.4.14.5,1.1.1.49,1.2.1.-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7857,C5NN18,5836.0,True,1,6.2.1.-,6.2.1.-,6.2.1.12,6.2.1.3,3.6.4.13,1.6.2.2,...,6.2.1.26,1.13.12.7,2.6.1.-,2.4.1.-,1.6.5.4,3.5.1.4,3.4.14.5,1.1.1.9,6.2.1.41,1.1.1.195
7930,Q8GZ29,5904.0,True,1,6.3.2.-,6.3.2.-,6.2.1.3,6.2.1.-,6.2.1.16,2.8.2.-,...,2.7.7.9,6.2.1.12,4.4.1.14,6.3.2.17,6.2.1.1,6.3.2.7,6.3.5.4,2.1.1.320,6.3.2.20,6.2.1.7
7948,I3R9K1,5955.0,True,1,6.3.3.3,6.3.3.3,1.3.7.7,6.3.4.2,6.3.5.11,2.7.1.-,...,3.6.1.15,2.7.7.2,6.3.2.8,2.7.8.28,6.3.5.9,3.4.19.15,2.3.1.8,1.1.1.47,4.1.1.-,2.7.1.78
7951,G0HV10,5977.0,True,1,6.3.4.2,6.3.4.2,6.3.5.5,6.3.5.2,6.3.5.11,2.3.1.31,...,2.7.7.27,1.20.4.4,1.20.4.-,2.3.1.8,2.3.1.46,3.4.19.9,1.3.99.37,2.2.1.10,1.7.1.16,4.2.1.155


## 7. Slice+esm33

In [139]:
train_esm33,test_esm33= bcommon.load_data_embedding(train=train_data, test=test, embedding_type=cfg.EMBEDDING_METHOD.get('esm33'))
train_X = train_esm33.iloc[:, 7:]
train_Y = pd.DataFrame(train_esm33['ec_label'])

test_X = test_esm33.iloc[:, 7:]
test_Y = pd.DataFrame(test_esm33['ec_label'])

cfg.FEATURE_NUM = 1280
#train
bcommon.prepare_slice_file(x_data=train_X, y_data=train_Y, x_file=cfg.DATADIR+'slice_train_x_esm33.txt', y_file=cfg.DATADIR+'slice_train_y_esm33.txt', ec_label_dict=dict_ec_label)
#test
bcommon.prepare_slice_file(x_data=test_X, y_data=test_Y, x_file=cfg.DATADIR+'slice_test_x_esm33.txt', y_file=cfg.DATADIR+'slice_test_y_esm33.txt', ec_label_dict=dict_ec_label)

slice files prepared success
slice files prepared success


In [140]:
print('step 6 trainning slice model')
#6. 训练Slice模型
btrain.train_ec_slice(trainX=cfg.DATADIR+'slice_train_x_esm33.txt', trainY=cfg.DATADIR+'slice_train_y_unirep.txt', modelPath=cfg.MODELDIR+'/slice_esm33')
slice_pred = btest.get_slice_res(slice_query_file=cfg.DATADIR+'slice_test_x_esm33.txt', model_path= cfg.MODELDIR+'/slice_esm33',dict_ec_label=dict_ec_label,test_set=test, res_file='/tmp/test.txt')

step 6 trainning slice model
 ./slice_train /home/shizhenkun/codebase/BioUniprot/data/benchmark/data/slice_train_x_esm33.txt /home/shizhenkun/codebase/BioUniprot/data/benchmark/data/slice_train_y_unirep.txt /home/shizhenkun/codebase/BioUniprot/data/benchmark/model/slice_esm33 -m 100 -c 300 -s 300 -k 700 -o 32 -t 32 -C 1 -f 0.000001 -siter 20 -stype 0 -q 0 
train finished
./slice_predict /home/shizhenkun/codebase/BioUniprot/data/benchmark/data/slice_test_x_esm33.txt /home/shizhenkun/codebase/BioUniprot/data/benchmark/model/slice_esm33 /tmp/test.txt -o 32 -b 0 -t 32 -q 0


In [141]:
s1res = test.iloc[:,np.r_[0:5]].merge(slice_pred, on='id', how='left')
s1res[s1res.ec_number==s1res.top0]

,id,ec_label,isemzyme,functionCounts,ec_number,top0,top1,top2,top3,top4,...,top10,top11,top12,top13,top14,top15,top16,top17,top18,top19
4852,A0A3B1EFP9,13.0,True,1,1.1.-.-,1.1.-.-,1.1.99.29,1.1.3.10,1.1.3.13,4.1.2.10,...,1.14.14.-,1.1.3.37,1.4.3.4,1.1.1.102,1.1.5.9,1.1.99.-,1.4.3.16,3.4.16.4,1.3.5.4,1.1.99.2
4870,A0A1Y0BRF3,18.0,True,1,1.1.1.-,1.1.1.-,1.1.1.1,1.3.1.-,5.1.3.2,1.3.1.112,...,1.1.1.170,1.4.3.3,1.1.1.349,1.1.1.283,1.2.1.44,1.1.1.270,1.3.1.3,1.8.-.-,1.3.1.77,1.14.13.237
4871,A0A1A9TAK5,18.0,True,1,1.1.1.-,1.1.1.-,1.1.-.-,1.1.1.184,1.1.1.100,1.1.1.1,...,1.3.1.38,1.1.1.30,1.1.1.138,1.1.1.206,3.1.1.1,1.1.1.47,1.6.5.5,1.1.1.141,1.1.1.62,3.6.4.3
4872,Q5M8N4,18.0,True,1,1.1.1.-,1.1.1.-,2.1.1.163,2.4.1.227,1.4.99.-,5.1.3.2,...,1.14.13.-,1.1.1.271,1.2.1.84,1.1.1.102,1.2.1.-,1.1.1.37,1.3.3.4,4.99.1.1,1.1.1.333,1.6.5.5
4873,M1VWN4,18.0,True,1,1.1.1.-,1.1.1.-,2.3.1.-,1.1.1.330,1.1.1.102,1.3.1.-,...,2.7.1.-,1.2.1.-,1.2.1.84,1.6.5.9,1.1.5.3,1.3.1.45,1.5.1.7,1.3.1.75,1.5.1.40,1.16.1.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7863,A0A2H5AIX5,5843.0,True,1,6.2.1.12,6.2.1.12,6.2.1.-,6.2.1.3,6.2.1.26,1.13.12.7,...,3.4.21.108,2.3.1.4,1.6.2.2,1.14.-.-,1.1.1.14,3.5.2.2,6.2.1.41,6.2.1.25,6.2.1.8,2.6.1.5
7930,Q8GZ29,5904.0,True,1,6.3.2.-,6.3.2.-,6.2.1.3,6.2.1.-,2.7.7.9,6.2.1.19,...,2.3.1.97,1.13.11.58,4.2.3.-,6.3.2.17,1.14.14.17,6.3.2.20,6.3.2.46,2.3.1.42,6.2.1.12,1.13.11.52
7948,I3R9K1,5955.0,True,1,6.3.3.3,6.3.3.3,2.7.1.130,6.3.5.11,6.3.4.2,5.3.3.2,...,1.3.7.7,2.7.8.28,3.6.3.16,1.5.1.36,2.7.7.68,3.1.1.96,3.4.19.15,1.1.1.47,1.1.1.169,1.7.7.2
7951,G0HV10,5977.0,True,1,6.3.4.2,6.3.4.2,6.3.5.2,2.3.1.31,6.3.5.5,6.3.5.11,...,2.1.1.298,5.4.2.12,2.2.1.10,3.6.3.16,6.4.1.3,1.3.99.37,1.1.1.388,1.17.7.1,4.1.1.99,1.20.4.-


In [144]:
s1res[s1res.functionCounts==1]

,id,ec_label,isemzyme,functionCounts,ec_number,top0,top1,top2,top3,top4,...,top10,top11,top12,top13,top14,top15,top16,top17,top18,top19
4454,Q2U0J6,1.0,True,1,1.-.-.-,1.14.-.-,1.14.14.-,2.3.1.-,1.14.13.-,1.1.1.330,...,1.14.14.23,2.4.1.80,1.14.21.-,1.14.13.176,3.4.24.59,1.14.13.175,1.14.13.174,1.14.21.10,4.2.1.92,1.14.14.29
4455,A0A2I6PIZ2,1.0,True,1,1.-.-.-,1.14.13.9,2.3.1.-,1.14.13.-,1.14.14.17,2.4.1.-,...,1.14.13.113,1.14.14.-,1.14.13.24,1.14.13.180,1.14.13.1,1.4.3.3,1.14.13.20,1.14.13.182,1.14.13.216,2.4.1.131
4456,D1MX87,1.0,True,1,1.-.-.-,1.14.-.-,1.14.14.-,1.14.13.-,1.14.13.70,3.1.-.-,...,2.3.1.-,1.3.3.6,1.14.19.-,1.14.21.10,1.14.14.29,4.2.1.121,2.8.3.21,4.2.1.92,1.14.99.-,1.14.19.41
4457,A0A075TR33,1.0,True,1,1.-.-.-,1.1.1.-,1.5.99.12,3.5.1.-,3.2.1.67,1.3.3.6,...,3.1.1.8,3.4.11.9,3.1.1.5,1.21.-.-,3.2.1.101,1.1.99.29,3.2.1.51,1.1.3.5,1.1.3.4,1.1.99.-
4458,Q2U0K0,1.0,True,1,1.-.-.-,1.14.-.-,1.14.14.-,2.3.1.-,1.14.13.175,1.14.13.174,...,2.4.1.131,1.14.21.10,2.4.99.12,1.1.1.1,2.7.1.12,2.6.1.57,2.4.1.129,2.4.1.117,1.3.3.6,1.14.13.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8028,Q9NTI2,6039.0,True,1,7.6.2.1,3.6.3.1,3.6.3.8,4.6.1.1,3.6.3.44,3.6.4.-,...,3.6.3.10,3.1.1.5,1.14.13.225,2.4.1.12,2.4.1.173,2.1.1.13,3.6.3.7,2.3.1.41,2.4.1.255,3.4.24.11
8029,D4AA47,6039.0,True,1,7.6.2.1,3.6.3.1,3.6.3.8,4.6.1.1,2.7.10.2,3.6.3.44,...,2.4.1.12,3.6.3.-,1.14.13.225,3.6.3.54,3.6.3.9,3.1.13.4,1.17.1.4,3.4.17.22,3.1.1.5,3.1.3.4
8030,F1RBC8,6040.0,True,1,7.6.2.4,3.6.4.12,3.1.-.-,3.6.3.44,2.4.2.28,3.6.-.-,...,3.4.-.-,2.7.11.2,1.14.13.70,1.6.2.4,2.7.11.4,3.4.24.84,1.3.99.30,3.4.23.-,2.1.1.309,1.3.8.9
8031,Q7JUN3,6040.0,True,1,7.6.2.4,3.6.4.12,3.1.-.-,3.6.3.44,1.14.13.9,3.6.5.-,...,1.6.5.9,3.6.-.-,2.4.99.18,5.3.3.5,1.2.1.84,3.4.24.84,3.1.1.97,1.22.-.-,4.2.1.47,1.13.11.27


In [148]:
fres = test_results_df.merge(s1res, on='id', how='left')

In [156]:
fres[fres.ec_pred_blast.isnull()]

,id,ec_number_x,ec_pred_blast,ec_label,isemzyme,functionCounts,ec_number_y,top0,top1,top2,...,top10,top11,top12,top13,top14,top15,top16,top17,top18,top19
1,P0DQF6,-,None,0.0,False,0,-,3.1.27.-,3.1.1.4,3.4.24.-,...,3.1.-.-,4.2.1.104,3.1.1.59,1.2.5.2,4.3.2.5,3.1.4.41,3.4.22.65,3.4.22.34,3.4.24.21,3.1.1.-
2,P0DQF5,-,None,0.0,False,0,-,3.4.-.-,3.4.21.-,3.4.22.-,...,3.1.1.3,2.1.-.-,1.23.5.1,3.1.1.59,1.1.1.23,3.4.24.21,3.4.24.13,5.4.99.5,3.1.4.41,3.1.3.33
3,P0DQF4,-,None,0.0,False,0,-,3.4.-.-,3.1.1.4,3.2.2.22,...,3.2.1.6,2.1.1.56,2.1.1.86,1.12.7.2,3.1.3.48,1.1.2.7,2.3.1.108,3.4.24.21,3.1.11.-,3.1.1.59
4,P0DQF3,-,None,0.0,False,0,-,3.1.27.-,3.4.-.-,3.1.3.48,...,3.4.24.21,3.4.21.5,1.8.3.2,2.4.1.183,3.1.4.41,2.4.99.18,3.1.1.59,3.2.1.6,2.4.1.244,2.4.2.26
5,P0DQF2,-,None,0.0,False,0,-,3.1.27.-,3.4.21.-,3.2.2.22,...,3.4.23.-,2.4.99.18,2.3.1.108,3.1.1.4,3.1.1.59,3.4.22.65,3.4.24.21,4.2.1.78,3.1.3.48,4.2.2.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7962,P0DTD9,6.5.1.-,None,6016.0,True,1,6.5.1.-,3.2.2.-,2.7.7.50,2.7.7.-,...,4.2.3.6,4.1.1.4,2.1.1.59,2.5.1.27,2.1.-.-,2.3.3.9,1.3.7.2,2.4.2.37,6.3.1.9,2.3.1.263
7977,C0HLQ4,7.1.2.2,None,6031.0,True,1,7.1.2.2,1.11.1.7,3.1.1.4,3.4.23.-,...,1.3.7.12,3.4.24.58,3.4.24.50,1.1.1.149,3.1.1.59,4.2.2.7,1.1.99.30,3.5.1.76,3.5.1.13,1.12.1.2
7981,Q8TSY2,7.2.1.-,None,6032.0,True,1,7.2.1.-,1.6.5.8,5.2.1.8,6.3.1.20,...,3.2.1.4,4.3.1.7,1.17.1.5,4.1.1.70,2.7.1.-,2.7.1.200,6.2.1.5,1.3.99.33,2.7.1.192,1.12.98.3
7986,H6LC27,7.2.1.2,None,6033.0,True,1,7.2.1.2,1.6.5.11,1.6.5.8,1.17.7.3,...,1.1.1.37,1.12.1.3,1.3.1.1,1.2.7.7,2.7.1.202,1.2.7.12,1.3.7.11,3.4.24.-,2.1.1.245,1.2.1.58


In [162]:
fres['ec_pred_slice']=fres.apply(lambda x: x.top0 if str(x.ec_pred_blast)=='None' else x.ec_pred_blast, axis=1)

In [170]:
fres[(fres.ec_number_x==fres.ec_pred_slice) & (fres.isemzyme) ]

,id,ec_number_x,ec_pred_blast,ec_label,isemzyme,functionCounts,ec_number_y,top0,top1,top2,...,top11,top12,top13,top14,top15,top16,top17,top18,top19,ec_pred_slice
4454,Q2U0J6,1.-.-.-,1.-.-.-,1.0,True,1,1.-.-.-,1.14.-.-,1.14.14.-,2.3.1.-,...,2.4.1.80,1.14.21.-,1.14.13.176,3.4.24.59,1.14.13.175,1.14.13.174,1.14.21.10,4.2.1.92,1.14.14.29,1.-.-.-
4455,A0A2I6PIZ2,1.-.-.-,1.-.-.-,1.0,True,1,1.-.-.-,1.14.13.9,2.3.1.-,1.14.13.-,...,1.14.14.-,1.14.13.24,1.14.13.180,1.14.13.1,1.4.3.3,1.14.13.20,1.14.13.182,1.14.13.216,2.4.1.131,1.-.-.-
4456,D1MX87,1.-.-.-,1.-.-.-,1.0,True,1,1.-.-.-,1.14.-.-,1.14.14.-,1.14.13.-,...,1.3.3.6,1.14.19.-,1.14.21.10,1.14.14.29,4.2.1.121,2.8.3.21,4.2.1.92,1.14.99.-,1.14.19.41,1.-.-.-
4457,A0A075TR33,1.-.-.-,1.-.-.-,1.0,True,1,1.-.-.-,1.1.1.-,1.5.99.12,3.5.1.-,...,3.4.11.9,3.1.1.5,1.21.-.-,3.2.1.101,1.1.99.29,3.2.1.51,1.1.3.5,1.1.3.4,1.1.99.-,1.-.-.-
4459,A2QBE8,1.-.-.-,1.-.-.-,1.0,True,1,1.-.-.-,1.14.-.-,2.3.1.-,1.14.14.-,...,1.3.3.6,1.14.13.176,1.14.21.-,1.14.13.78,1.1.1.332,1.14.99.-,2.4.-.-,4.2.1.121,2.4.1.-,1.-.-.-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7952,O66164,6.3.4.3,6.3.4.3,5983.0,True,1,6.3.4.3,6.3.4.3,6.3.4.2,2.4.2.4,...,6.1.1.1,6.2.1.9,3.6.4.9,4.4.1.24,3.6.5.-,4.2.1.42,1.1.1.88,6.3.3.3,1.1.1.409,6.3.4.3
7958,A0A0H3JRU9,6.4.1.1,6.4.1.1,6005.0,True,1,6.4.1.1,6.3.5.5,6.4.1.1,2.7.7.7,...,2.4.1.15,1.7.99.4,2.1.1.13,3.4.21.110,1.17.1.4,6.3.4.6,1.4.1.14,3.6.5.3,6.3.2.-,6.4.1.1
7959,Q168G2,6.4.1.3,6.4.1.3,6008.0,True,1,6.4.1.3,6.4.1.2,6.4.1.3,6.3.4.2,...,4.1.3.27,4.2.1.149,2.1.3.1,6.4.1.-,2.5.1.6,4.1.1.70,4.2.1.17,2.3.1.8,5.1.3.4,6.4.1.3
7960,Q5LUF3,6.4.1.3,6.4.1.3,6008.0,True,1,6.4.1.3,6.2.1.5,6.4.1.4,6.4.1.3,...,2.3.1.61,4.1.3.27,3.6.3.20,2.7.9.2,2.7.4.6,5.4.99.21,1.13.11.25,2.1.3.1,3.6.3.21,6.4.1.3


In [175]:
fres[(fres.ec_number_x==fres.top1) & (fres.isemzyme) ]

,id,ec_number_x,ec_pred_blast,ec_label,isemzyme,functionCounts,ec_number_y,top0,top1,top2,...,top11,top12,top13,top14,top15,top16,top17,top18,top19,ec_pred_slice
4875,P9WEW9,1.1.1.-,1.1.1.391,18.0,True,1,1.1.1.-,1.1.1.1,1.1.1.-,1.1.1.100,...,1.1.1.10,1.3.1.38,1.5.1.33,1.1.1.304,1.1.1.184,1.14.14.-,1.1.1.385,1.1.1.289,1.5.1.2,1.1.1.391
4878,W6QIM3,1.1.1.-,1.1.1.332,18.0,True,1,1.1.1.-,1.1.1.1,1.1.1.-,2.3.1.-,...,1.1.1.304,1.1.1.250,1.3.1.87,1.5.1.33,1.5.1.34,1.1.1.184,1.1.1.331,1.1.1.352,1.3.1.38,1.1.1.332
4879,A0A348AXY0,1.1.1.-,1.1.1.28,18.0,True,1,1.1.1.-,1.1.1.290,1.1.1.-,1.1.1.28,...,1.1.1.95,1.1.1.237,1.20.1.1,1.1.1.310,1.4.1.1,1.4.1.23,1.4.1.20,3.5.1.10,6.2.1.9,1.1.1.28
4893,A0A1J0HSP5,1.1.1.-,None,18.0,True,1,1.1.1.-,1.1.1.1,1.1.1.-,1.3.1.9,...,1.3.1.45,1.1.1.349,1.2.1.-,1.3.1.56,1.1.1.320,2.1.2.2,2.6.1.-,4.1.1.-,1.5.1.40,1.1.1.1
4918,Q7SHI4,1.1.1.-,1.1.1.-,18.0,True,1,1.1.1.-,1.1.1.1,1.1.1.-,1.1.1.100,...,1.1.1.141,1.3.1.56,1.1.-.-,1.1.1.36,1.3.1.38,1.1.1.304,1.1.1.331,1.5.1.33,1.1.1.184,1.1.1.-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7937,S3DQP3,6.3.2.-,6.3.2.-,5904.0,True,1,6.3.2.-,2.3.1.-,6.3.2.-,6.2.1.-,...,1.1.1.-,6.1.1.20,2.3.1.244,2.3.1.144,2.1.1.116,5.1.1.-,1.14.13.174,6.2.1.46,1.14.13.177,6.3.2.-
7938,F9X9V1,6.3.2.-,6.3.2.-,5904.0,True,1,6.3.2.-,2.3.1.-,6.3.2.-,6.2.1.-,...,2.6.1.-,3.4.24.59,3.1.4.37,4.4.-.-,1.1.1.332,1.16.1.-,3.4.22.49,2.3.1.84,1.16.1.9,6.3.2.-
7958,A0A0H3JRU9,6.4.1.1,6.4.1.1,6005.0,True,1,6.4.1.1,6.3.5.5,6.4.1.1,2.7.7.7,...,2.4.1.15,1.7.99.4,2.1.1.13,3.4.21.110,1.17.1.4,6.3.4.6,1.4.1.14,3.6.5.3,6.3.2.-,6.4.1.1
7959,Q168G2,6.4.1.3,6.4.1.3,6008.0,True,1,6.4.1.3,6.4.1.2,6.4.1.3,6.3.4.2,...,4.1.3.27,4.2.1.149,2.1.3.1,6.4.1.-,2.5.1.6,4.1.1.70,4.2.1.17,2.3.1.8,5.1.3.4,6.4.1.3
